In [6]:
#pip install dash

## Analise

- Qual produto gerou a maior receita total ao longo do período analisado?

Entender quais produtos são mais lucrativos ajuda a focar em estratégias de vendas para esses itens.

- Quais são os produtos com maior taxa de cancelamento de pedidos e quais podem ser as razões?

Identificar produtos com alta taxa de cancelamento pode apontar para problemas de qualidade, preços, ou outras questões que precisam ser resolvidas.

- Os clientes membros do programa de fidelidade compram com mais frequência ou gastam mais por transação em comparação aos não membros?

Essa análise ajuda a avaliar a eficácia do programa de fidelidade e se ele realmente impulsiona as vendas.

- Quais são os métodos de pagamento mais utilizados e como eles influenciam o ticket médio das compras?

Pode revelar preferências dos clientes e ajudar a ajustar as opções de pagamento para maximizar a receita.

- Existe uma relação entre a faixa etária dos clientes e o tipo de produto comprado?

Isso pode auxiliar no direcionamento de campanhas de marketing específicas para diferentes grupos demográficos.

- Qual é o impacto do tipo de envio (Standard, Overnight, Express) na avaliação de satisfação do cliente?

Entender como o tempo de entrega afeta a satisfação é essencial para otimizar a experiência do cliente.

- Qual é a época do ano (ou mês) com o maior volume de vendas?

Identificar padrões sazonais permite planejar melhor promoções e estoques.

- Quais produtos têm maior taxa de compra de "Add-ons" e quais combinações são mais populares?

Isso ajuda a criar estratégias de vendas cruzadas (cross-sell) e aumentar o valor do ticket médio.

- Qual é o perfil (idade, gênero, fidelidade) dos clientes que mais gastam?

Conhecer o perfil dos seus clientes mais valiosos permite criar campanhas mais direcionadas e eficazes.

In [7]:
import dash
from dash import dcc, html
import plotly.express as px
import pandas as pd

In [8]:
df = pd.read_csv('/content/Electronic_sales_Sep2023-Sep2024.csv')

In [9]:
df.head()

,Customer ID,Age,Gender,Loyalty Member,Product Type,SKU,Rating,Order Status,Payment Method,Total Price,Unit Price,Quantity,Purchase Date,Shipping Type,Add-ons Purchased,Add-on Total
0,1000,53,Male,No,Smartphone,SKU1004,2,Cancelled,Credit Card,5538.33,791.19,7,2024-03-20,Standard,"Accessory,Accessory,Accessory",40.21
1,1000,53,Male,No,Tablet,SKU1002,3,Completed,Paypal,741.09,247.03,3,2024-04-20,Overnight,Impulse Item,26.09
2,1002,41,Male,No,Laptop,SKU1005,3,Completed,Credit Card,1855.84,463.96,4,2023-10-17,Express,NaN,0.00
3,1002,41,Male,Yes,Smartphone,SKU1004,2,Completed,Cash,3164.76,791.19,4,2024-08-09,Overnight,"Impulse Item,Impulse Item",60.16
4,1003,75,Male,Yes,Smartphone,SKU1001,5,Completed,Cash,41.50,20.75,2,2024-05-21,Express,Accessory,35.56


In [10]:
df.describe()

,Customer ID,Age,Rating,Total Price,Unit Price,Quantity,Add-on Total
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,10483.526550,48.994100,3.093950,3180.133419,578.631867,5.485550,62.244848
std,5631.732525,18.038745,1.223764,2544.978675,312.274076,2.870854,58.058431
min,1000.000000,18.000000,1.000000,20.750000,20.750000,1.000000,0.000000
25%,5478.000000,33.000000,2.000000,1139.680000,361.180000,3.000000,7.615000
50%,10499.500000,49.000000,3.000000,2534.490000,463.960000,5.000000,51.700000
75%,15504.000000,65.000000,4.000000,4639.600000,791.190000,8.000000,93.842500
max,19998.000000,80.000000,5.000000,11396.800000,1139.680000,10.000000,292.770000


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Customer ID        20000 non-null  int64  
 1   Age                20000 non-null  int64  
 2   Gender             19999 non-null  object 
 3   Loyalty Member     20000 non-null  object 
 4   Product Type       20000 non-null  object 
 5   SKU                20000 non-null  object 
 6   Rating             20000 non-null  int64  
 7   Order Status       20000 non-null  object 
 8   Payment Method     20000 non-null  object 
 9   Total Price        20000 non-null  float64
 10  Unit Price         20000 non-null  float64
 11  Quantity           20000 non-null  int64  
 12  Purchase Date      20000 non-null  object 
 13  Shipping Type      20000 non-null  object 
 14  Add-ons Purchased  15132 non-null  object 
 15  Add-on Total       20000 non-null  float64
dtypes: float64(3), int64(4

Qual produto gerou a maior receita total ao longo do período analisado?

In [12]:
receita_por_produto = df.groupby('Product Type')['Unit Price'].sum()

# Encontrar o produto com a maior receita
produto_maior_receita = receita_por_produto.idxmax()
maior_receita = receita_por_produto.max()

print(f"O produto com maior receita total é {produto_maior_receita} com R${maior_receita:.2f} em vendas")

O produto com maior receita total é Smartphone com R$3902921.89 em vendas


Quais são os produtos com maior taxa de cancelamento de pedidos e quais podem ser as razões?

In [13]:
cancelamentos_por_produto = df[df['Order Status'] == 'Cancelled'].groupby('Product Type').size()
total_por_produto = df.groupby('Product Type').size()
taxa_cancelamento = (cancelamentos_por_produto / total_por_produto) * 100

# Encontrando os produtos com as maiores taxas de cancelamento
produtos_maior_cancelamento = taxa_cancelamento.sort_values(ascending=False)

print("Produtos com maior taxa de cancelamento de pedidos:")
produtos_maior_cancelamento

# Possíveis razões para o cancelamento (requer análise mais aprofundada dos dados):
# - Problemas de qualidade do produto:  Verificar se há relatos de defeitos ou problemas com produtos específicos.
# - Preços altos em comparação com a concorrência: Analisar preços praticados pela concorrência para produtos similares.
# - Descrições enganosas ou falta de informações relevantes: Revisar as descrições dos produtos e se há clareza na comunicação das características dos produtos.
# - Problemas com a entrega: Analisar os dados de entregas para verificar atrasos, extravios, ou danos no transporte.
# - Problemas com o atendimento ao cliente: Verificar se houve reclamações sobre o atendimento durante o processo de compra ou pós venda.
# - Competidores oferecendo preços ou condições melhores: Monitorar a concorrência para identificar ofertas mais atraentes.
# - Mudança de ideia do cliente:  É normal que ocorram cancelamentos por arrependimento da compra.
# - Erros no processo de compra: Analisar se houve erros no site ou no processo de checkout que levem ao cancelamento.

Produtos com maior taxa de cancelamento de pedidos:


,0
Product Type,
Tablet,33.114035
Smartphone,33.021077
Smartwatch,32.994408
Laptop,32.393657
Headphones,32.322228


Os clientes membros do programa de fidelidade compram com mais frequência ou gastam mais por transação em comparação aos não membros?

In [14]:
# Agrupar os dados por membros e não-membros e calcular a frequência de compras e o gasto médio por transação
loyalty_analysis = df.groupby('Loyalty Member').agg(
    Total_Sales=('Total Price', 'sum'),
    Avg_Spend_Per_Transaction=('Total Price', 'mean'),
    Total_Transactions=('Customer ID', 'count')
).reset_index()


loyalty_analysis

,Loyalty Member,Total_Sales,Avg_Spend_Per_Transaction,Total_Transactions
0,No,49976751.24,3191.974915,15657
1,Yes,13625917.13,3137.443502,4343


Quais são os métodos de pagamento mais utilizados e como eles influenciam o ticket médio das compras?

In [15]:
# Analisar os métodos de pagamento e como eles influenciam o ticket médio das compras

# Agrupar os dados por método de pagamento para calcular o total de vendas e o ticket médio por transação
payment_analysis = df.groupby('Payment Method').agg(
    Total_Sales=('Total Price', 'sum'),
    Avg_Spend_Per_Transaction=('Total Price', 'mean'),
    Total_Transactions=('Customer ID', 'count')
).reset_index()

# Ordenar pelo total de vendas
payment_analysis = payment_analysis.sort_values(by='Total_Sales', ascending=False)
payment_analysis

,Payment Method,Total_Sales,Avg_Spend_Per_Transaction,Total_Transactions
2,Credit Card,18913772.92,3223.206019,5868
0,Bank Transfer,12698262.09,3766.912516,3371
4,PayPal,12500062.91,3806.352896,3284
3,Debit Card,6736217.54,2726.109891,2471
5,Paypal,6498781.08,2585.036229,2514
1,Cash,6255571.83,2510.261569,2492


Existe uma relação entre a faixa etária dos clientes e o tipo de produto comprado?

In [16]:
# Análise da relação entre a faixa etária dos clientes e o tipo de produto comprado

# Definir faixas etárias para facilitar a análise
bins = [18, 30, 40, 50, 60, 70, 80, 90]
labels = ['18-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90']
df['Age Group'] = pd.cut(df['Age'], bins=bins, labels=labels)

# Agrupar os dados por faixa etária e tipo de produto para calcular o número de transações
age_product_analysis = df.groupby(['Age Group', 'Product Type']).size().reset_index(name='Transaction Count')

# Ordenar para facilitar a visualização
age_product_analysis.sort_values(by=['Age Group', 'Transaction Count'], ascending=[True, False], inplace=True)
age_product_analysis


<ipython-input-16-732ce4fd802e>:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  age_product_analysis = df.groupby(['Age Group', 'Product Type']).size().reset_index(name='Transaction Count')


,Age Group,Product Type,Transaction Count
2,18-30,Smartphone,1084
4,18-30,Tablet,781
1,18-30,Laptop,759
3,18-30,Smartwatch,741
0,18-30,Headphones,403
7,31-40,Smartphone,984
6,31-40,Laptop,671
8,31-40,Smartwatch,624
9,31-40,Tablet,624
5,31-40,Headphones,346


Qual é o impacto do tipo de envio (Standard, Overnight, Express) na avaliação de satisfação do cliente?

In [17]:
# Análise do impacto do tipo de envio na avaliação de satisfação do cliente

# Agrupar os dados por tipo de envio para calcular a média de avaliação (Rating)
shipping_rating_analysis = df.groupby('Shipping Type').agg(
    Avg_Rating=('Rating', 'mean'),
    Total_Transactions=('Customer ID', 'count')
).reset_index()

# Ordenar pelo rating médio para facilitar a análise
shipping_rating_analysis.sort_values(by='Avg_Rating', ascending=False, inplace=True)
shipping_rating_analysis

,Shipping Type,Avg_Rating,Total_Transactions
1,Express,3.198455,3366
2,Overnight,3.179029,3357
4,Standard,3.105576,6725
0,Expedited,2.990220,3272
3,Same Day,2.979268,3280


Qual é a época do ano (ou mês) com o maior volume de vendas?

In [18]:
# Analisar o volume de vendas por mês para identificar a época do ano com maior volume de transações

# Converter a coluna 'Purchase Date' para datetime e extrair o mês
df['Purchase Date'] = pd.to_datetime(df['Purchase Date'])
df['Month'] = df['Purchase Date'].dt.strftime('%Y-%m')

# Agrupar os dados por mês e calcular o total de vendas
monthly_sales_analysis = df.groupby('Month').agg(
    Total_Sales=('Total Price', 'sum'),
    Total_Transactions=('Customer ID', 'count')
).reset_index()

# Ordenar por total de vendas em ordem decrescente para identificar os meses com maior volume
monthly_sales_analysis.sort_values(by='Total_Sales', ascending=False, inplace=True)
monthly_sales_analysis

,Month,Total_Sales,Total_Transactions
8,2024-05,6709042.93,2048
11,2024-08,6706118.61,2018
9,2024-06,6668633.59,1999
4,2024-01,6620172.47,2049
10,2024-07,6535129.52,1989
7,2024-04,6418253.62,1932
6,2024-03,6324367.84,1957
5,2024-02,5733696.06,1818
12,2024-09,5037691.12,1492
1,2023-10,2318466.35,911


Quais produtos têm maior taxa de compra de "Add-ons" e quais combinações são mais populares?

In [19]:
# Analisar a taxa de compra de "Add-ons" por tipo de produto

# Criar uma coluna indicando se "Add-ons" foram comprados (1 se houver, 0 se não)
df['Add-ons Purchased'] = df['Add-ons Purchased'].fillna('')
df['Has_Addons'] = df['Add-ons Purchased'].apply(lambda x: 1 if x.strip() != '' else 0)

# Agrupar os dados por tipo de produto para calcular a taxa de compra de add-ons
addon_product_analysis = df.groupby('Product Type').agg(
    Total_Transactions=('Customer ID', 'count'),
    Transactions_with_Addons=('Has_Addons', 'sum')
).reset_index()

# Calcular a taxa de compra de add-ons
addon_product_analysis['Addon_Purchase_Rate'] = addon_product_analysis['Transactions_with_Addons'] / addon_product_analysis['Total_Transactions'] * 100

# Ordenar pela taxa de compra de add-ons em ordem decrescente
addon_product_analysis.sort_values(by='Addon_Purchase_Rate', ascending=False, inplace=True)

# Analisar as combinações mais populares de "Add-ons"
addon_combinations = df['Add-ons Purchased'].value_counts().reset_index()
addon_combinations.columns = ['Add-on Combination', 'Count']

# Exibir os resultados
addon_product_analysis, addon_combinations.head(10)


(  Product Type  Total_Transactions  Transactions_with_Addons  \
 3   Smartwatch                3934                      2992   
 0   Headphones                2011                      1528   
 4       Tablet                4104                      3117   
 1       Laptop                3973                      3003   
 2   Smartphone                5978                      4492   
 
    Addon_Purchase_Rate  
 3            76.054906  
 0            75.982098  
 4            75.950292  
 1            75.585200  
 2            75.142188  ,
                Add-on Combination  Count
 0                                   4868
 1               Extended Warranty   1701
 2                       Accessory   1680
 3                    Impulse Item   1645
 4  Extended Warranty,Impulse Item    311
 5          Accessory,Impulse Item    307
 6          Impulse Item,Accessory    305
 7      Impulse Item, Impulse Item    301
 8       Impulse Item,Impulse Item    299
 9         Impulse Item, Access

In [20]:
addon_product_analysis

,Product Type,Total_Transactions,Transactions_with_Addons,Addon_Purchase_Rate
3,Smartwatch,3934,2992,76.054906
0,Headphones,2011,1528,75.982098
4,Tablet,4104,3117,75.950292
1,Laptop,3973,3003,75.585200
2,Smartphone,5978,4492,75.142188


In [21]:
addon_combinations

,Add-on Combination,Count
0,,4868
1,Extended Warranty,1701
2,Accessory,1680
3,Impulse Item,1645
4,"Extended Warranty,Impulse Item",311
...,...,...
71,"Accessory, Extended Warranty, Accessory",80
72,"Accessory,Extended Warranty,Extended Warranty",78
73,"Extended Warranty, Extended Warranty, Accessory",76
74,"Accessory,Impulse Item,Impulse Item",71


Qual é o perfil (idade, gênero, fidelidade) dos clientes que mais gastam?

In [22]:
# Análise do perfil dos clientes que mais gastam com base em idade, gênero e fidelidade

# Agrupar os dados por idade, gênero e status de fidelidade para calcular o total de vendas
customer_profile_analysis = df.groupby(['Age', 'Gender', 'Loyalty Member']).agg(
    Total_Spent=('Total Price', 'sum')
).reset_index()

# Ordenar pelos clientes que mais gastaram em ordem decrescente
customer_profile_analysis.sort_values(by='Total_Spent', ascending=False, inplace=True)

# Exibir os 10 perfis de clientes que mais gastaram
top_spending_profiles = customer_profile_analysis.head(10)
top_spending_profiles

,Age,Gender,Loyalty Member,Total_Spent
182,63,Male,No,537396.48
240,78,Female,No,529882.50
210,70,Male,No,526165.03
24,24,Female,No,525080.99
68,35,Female,No,500593.82
72,36,Female,No,498625.85
100,43,Female,No,495626.67
214,71,Male,No,493894.42
130,50,Male,No,493004.97
140,53,Female,No,477097.20


In [23]:
# Carregar o dataset
data = pd.read_csv('/content/Electronic_sales_Sep2023-Sep2024.csv')

# Funções para análises

# 1. Qual produto gerou a maior receita total?
def top_revenue_product(data):
    revenue_by_product = data.groupby('Product Type').agg(
        Total_Revenue=('Total Price', 'sum')
    ).reset_index()
    return revenue_by_product.sort_values(by='Total_Revenue', ascending=False)

# 2. Produtos com maior taxa de cancelamento de pedidos
def cancellation_rate(data):
    canceled_orders = data[data['Order Status'] == 'Cancelled']
    cancellation_by_product = canceled_orders.groupby('Product Type').size().reset_index(name='Cancellations')
    return cancellation_by_product.sort_values(by='Cancellations', ascending=False)

# 3. Comparação entre clientes membros e não membros do programa de fidelidade
def loyalty_analysis(data):
    loyalty_data = data.groupby('Loyalty Member').agg(
        Total_Sales=('Total Price', 'sum'),
        Avg_Spend=('Total Price', 'mean'),
        Total_Transactions=('Customer ID', 'count')
    ).reset_index()
    return loyalty_data

# 4. Análise dos métodos de pagamento
def payment_analysis(data):
    payment_data = data.groupby('Payment Method').agg(
        Total_Sales=('Total Price', 'sum'),
        Avg_Spend=('Total Price', 'mean')
    ).reset_index()
    return payment_data

# 5. Relação entre faixa etária e tipo de produto comprado
def age_product_relation(data):
    data['Age Group'] = pd.cut(data['Age'], bins=[18, 30, 40, 50, 60, 70, 80, 90], labels=['18-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90'])
    age_product_data = data.groupby(['Age Group', 'Product Type']).size().reset_index(name='Transactions')
    return age_product_data

# 6. Impacto do tipo de envio na satisfação do cliente
def shipping_rating(data):
    shipping_data = data.groupby('Shipping Type').agg(
        Avg_Rating=('Rating', 'mean'),
        Total_Transactions=('Customer ID', 'count')
    ).reset_index()
    return shipping_data

# 7. Análise sazonal de vendas
def sales_by_month(data):
    data['Purchase Date'] = pd.to_datetime(data['Purchase Date'])
    data['Month'] = data['Purchase Date'].dt.strftime('%Y-%m')
    monthly_sales = data.groupby('Month').agg(Total_Sales=('Total Price', 'sum')).reset_index()
    return monthly_sales

# 8. Taxa de compra de add-ons por produto
def addons_analysis(data):
    data['Has_Addons'] = data['Add-ons Purchased'].notna()
    addons_by_product = data.groupby('Product Type').agg(
        Total_Transactions=('Customer ID', 'count'),
        Transactions_with_Addons=('Has_Addons', 'sum')
    ).reset_index()
    addons_by_product['Addon_Purchase_Rate'] = (addons_by_product['Transactions_with_Addons'] / addons_by_product['Total_Transactions']) * 100
    return addons_by_product

# 9. Perfil dos clientes que mais gastam
def top_customers_profile(data):
    customer_profile = data.groupby(['Age', 'Gender', 'Loyalty Member']).agg(
        Total_Spent=('Total Price', 'sum')
    ).reset_index()
    return customer_profile.sort_values(by='Total_Spent', ascending=False).head(10)

# Inicializar a aplicação Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Relatório de Vendas de E-commerce"),

    # Gráfico 1: Receita por Produto
    html.H2("Produtos com Maior Receita Total"),
    dcc.Graph(
        figure=px.bar(top_revenue_product(data), x='Product Type', y='Total_Revenue')
    ),

    # Gráfico 2: Produtos com Maior Taxa de Cancelamento
    html.H2("Produtos com Maior Taxa de Cancelamento"),
    dcc.Graph(
        figure=px.bar(cancellation_rate(data), x='Product Type', y='Cancellations')
    ),

    # Gráfico 3: Análise de Fidelidade
    html.H2("Comparação entre Clientes Membros e Não-Membros"),
    dcc.Graph(
        figure=px.bar(loyalty_analysis(data), x='Loyalty Member', y='Total_Sales')
    ),

    # Gráfico 4: Métodos de Pagamento
    html.H2("Métodos de Pagamento Mais Utilizados"),
    dcc.Graph(
        figure=px.bar(payment_analysis(data), x='Payment Method', y='Total_Sales')
    ),

    # Gráfico 5: Relação Faixa Etária x Tipo de Produto
    html.H2("Faixa Etária e Tipo de Produto"),
    dcc.Graph(
        figure=px.bar(age_product_relation(data), x='Age Group', y='Transactions', color='Product Type')
    ),

    # Gráfico 6: Satisfação por Tipo de Envio
    html.H2("Satisfação por Tipo de Envio"),
    dcc.Graph(
        figure=px.bar(shipping_rating(data), x='Shipping Type', y='Avg_Rating')
    ),

    # Gráfico 7: Vendas por Mês
    html.H2("Volume de Vendas por Mês"),
    dcc.Graph(
        figure=px.line(sales_by_month(data), x='Month', y='Total_Sales')
    ),

    # Gráfico 8: Taxa de Compra de Add-ons
    html.H2("Taxa de Compra de Add-ons por Produto"),
    dcc.Graph(
        figure=px.bar(addons_analysis(data), x='Product Type', y='Addon_Purchase_Rate')
    ),

    # Gráfico 9: Perfil dos Clientes que Mais Gastam
    html.H2("Clientes que Mais Gastam"),
    dcc.Graph(
        figure=px.bar(top_customers_profile(data), x='Age', y='Total_Spent', color='Gender')
    ),
])

if __name__ == '__main__':
    app.run_server(debug=True)

<ipython-input-23-d0c1157761df>:39: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



<IPython.core.display.Javascript object>